# PRACTICA 6: QUICK START with REDIS SINGLE NODE

Primero importamos la librería Redis.

In [2]:
import redis

Creamos una conexión al servidor de Redis.

In [4]:
r = redis.Redis(host='127.0.0.1', port=16379)

Reiniciamos la BBDD de Redis para eliminar los datasets existentes.

In [5]:
r.flushall()

True

A modo de prueba, guardamos un par de claves/valores en la base de datos. Pra el caso de  _Strings_:

In [7]:
r.set('David', 'el mejor')
r.get('David')

b'el mejor'